In [1]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [2]:
data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/train.csv')
test_data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/test.csv')

In [3]:
data['handle'] = data['handle'].replace({'realDonaldTrump':1, 'HillaryClinton':0})
data['tweet'] = data['tweet'].apply(lambda x: x.lower())
data['tweet'] = data['tweet'].apply((lambda x: re.sub(r"http\S+", '', x)))
data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
test_data['tweet'] = test_data['tweet'].apply(lambda x: x.lower())
test_data['tweet'] = test_data['tweet'].apply((lambda x: re.sub(r"http\S+", '', x)))
test_data['tweet'] = test_data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [4]:
X_train, X_val, y_train, y_val = train_test_split(data['tweet'], data['handle'],  test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(4000,) (4000,)
(1000,) (1000,)


In [5]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_val_counts = count_vect.transform(X_val)
X_val_tfidf = tfidf_transformer.transform(X_val_counts)

In [7]:
hyperVec = [1e10,1e9,1e8,1e7,1e6,1e5,1e4,1e3,500,400,300,200,150,1e2,90,80,70,60,50,40,30,20,1e1,9,8,7,6,5,4,3,2,1e0,0.5,0.2]
loss_dic = {}
for C in hyperVec:
    lf = LogisticRegression(C=C)
    lf.fit(X_train_tfidf, y_train)
    y_val_predicted = lf.predict(X_val_tfidf)
    loss_dic[C] = log_loss(y_val_predicted, y_val)
loss_dic

{0.2: 3.937483677216804,
 0.5: 3.6266258940908758,
 1.0: 3.5920823201113836,
 2: 3.246685760590544,
 3: 3.0739878806288408,
 4: 3.0739878806288408,
 5: 3.0394491042339298,
 6: 2.970370751846678,
 7: 2.970369952249248,
 8: 2.970369952249248,
 9: 2.935830376256907,
 10.0: 2.935830376256907,
 20: 2.866751224272225,
 30: 2.9703675534569576,
 40: 2.9012884014722755,
 50: 2.8667480258825044,
 60: 2.8667480258825044,
 70: 2.797670473092683,
 80: 2.797669673495253,
 90: 2.797670473092683,
 100.0: 2.901288401472276,
 150: 2.8322108486824544,
 200: 2.7976720722875434,
 300: 2.8667496250773645,
 400: 2.9358279774646165,
 500: 2.9703675534569576,
 1000.0: 2.9703675534569576,
 10000.0: 3.1776010114238518,
 100000.0: 3.2121405874161923,
 1000000.0: 3.1085226590366,
 10000000.0: 3.1430622350289403,
 100000000.0: 3.1430622350289403,
 1000000000.0: 3.1430622350289403,
 10000000000.0: 3.1430622350289403}

In [8]:
X_counts = count_vect.fit_transform(data['tweet'])
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [9]:
C = 200 
lf = LogisticRegression(C=C)
lf.fit(X_tfidf, data['handle'])

LogisticRegression(C=200, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
X_Test_counts = count_vect.transform(test_data['tweet'])
X_Test_tfidf = tfidf_transformer.transform(X_Test_counts)

In [11]:
predicted = lf.predict(X_Test_tfidf)
predict_probabs = lf.predict_proba(X_Test_tfidf) 

In [12]:
predicted.shape

(1444,)

In [13]:
filename = "/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/submission_bow_logistic_3.csv"
with open(filename,"w+") as outputfile:
    outputfile.write("id,realDonaldTrump,HillaryClinton\n")
    for i in range(predict_probabs.shape[0]):
        hillary, donald = predict_probabs[i][0], predict_probabs[i][1]
        outputfile.write(str(i)+","+str(donald)+","+str(hillary)+"\n")